Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [1]:
from random import choices
from random import random
from random import randint
from copy import deepcopy
import numpy as np
import math

import lab9_lib

In [2]:
POP_SIZE = 10
PARENT_SIZE = math.ceil(POP_SIZE / 4)
GENOME_SIZE = 200
PROBLEM_SIZE = 200

In [3]:
class Individual:
    genome:list
    fitness:float
    
    def __init__(self, genome:list=None):
        if genome is None:
            genome = choices([0, 1], k=GENOME_SIZE)     
        self.genome=genome
        self.fitness=fitness(genome)
        pass
    
    def mutate(self):
        child = deepcopy(self)
        nbmute:int = math.ceil((random()/self.fitness)*GENOME_SIZE*0.1)
        if nbmute > math.ceil(GENOME_SIZE*0.5)-1:
            nbmute = math.ceil(GENOME_SIZE*0.5)-1
        for i in range(0,nbmute):
            child.genome[randint(0,GENOME_SIZE-1)]^=1
        child.fitness=fitness(child.genome)
        return child

    def recomb(self, parent2):
        child = deepcopy(self)
        rangelswap:int = math.ceil((random()/self.fitness)*GENOME_SIZE*0.1)
        if rangelswap > math.ceil(GENOME_SIZE*0.75)-1 :
            rangelswap = math.ceil(GENOME_SIZE*0.75)-1
        child.genome = child.genome[0:GENOME_SIZE-rangelswap] + parent2.genome[GENOME_SIZE-rangelswap:]
        child.fitness=fitness(child.genome)
        return child



class Parent:
    parentList:[Individual]
    
    def __init__(self,pop:[Individual]=None):
        if pop is None:
            pop = [Individual() for i in range(0, POP_SIZE)]
        self.setParentList(pop)
        pass
    
    def getBestPop(self,pop:[Individual]=None)->[Individual]:
        if pop is None:
            return self.parentList[0].fitness
        return sorted(pop, key=lambda x:x.fitness,reverse=True)[0:PARENT_SIZE]
    
    def createNewPop(self)->[Individual]:
        newpop=[]
        newpop = newpop + self.parentList
        for i in range(len(self.parentList), POP_SIZE):
            choosenparent = choices(self.parentList, [j.fitness for j in self.parentList])[0]
            newpop.append(
                choices(
                    [choosenparent.mutate(), choosenparent.recomb(
                        choices(
                            [j for j in self.parentList if j != choosenparent],
                            [j.fitness for j in self.parentList if j != choosenparent])[0])], 
                    [choosenparent.fitness, 1 - choosenparent.fitness])[0])
        return newpop

    def setParentList(self, pop:[Individual]):
        self.parentList = self.getBestPop(pop)
        pass



In [4]:
fitness = lab9_lib.make_problem(PROBLEM_SIZE)

population = [Individual() for i in range(0, POP_SIZE)]
listofparent = Parent(population)
while listofparent.getBestPop() != 1:
    population = listofparent.createNewPop()
    listofparent.setParentList(population)

print("OK found it")
print(fitness.calls)


OK found it
48520
